In [4]:
!pip install adapter-transformers
!pip install datasets

In [5]:
cd drive/MyDrive/CAMemBERT2/

/content/drive/MyDrive/CAMemBERT2


In [6]:
from transformers import AutoModelWithHeads,AutoConfig,Trainer,TrainingArguments,EarlyStoppingCallback
import numpy as np
import pdb
from sklearn.metrics import cohen_kappa_score,f1_score,accuracy_score,fbeta_score,classification_report
from scipy.stats import spearmanr
from trainer_utils import get_dataset
import torch
import pandas as pd
from transformers.adapters.composition import Fuse

dataset_obj,dataset_dict = get_dataset(['ner'],max_length=512)
# prepare for model
dataset_dict = dataset_dict.remove_columns(['dataset','scores'])

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Fa

...

  0%|          | 0/266 [00:00<?, ?ex/s]

  0%|          | 0/243 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [9]:
lr = 5e-4
bs = 8
config_ = AutoConfig.from_pretrained('distilroberta-base',num_labels=9)
model_ = AutoModelWithHeads.from_pretrained('distilroberta-base',config=config_)
model_.load_adapter('/content/drive/MyDrive/CAMemBERT2/adapters/ged', load_as="ged", with_head=False)
adapter_setup_ = Fuse('ged')
model_.add_fusion(adapter_setup_)
model_.add_tagging_head(
            'ner_fuse',
            num_labels=9,
            overwrite_ok=True
        )
model_.set_active_adapters(adapter_setup_)
model_.train_fusion(adapter_setup_)

def compute_metrics(p):
    logits, labels = p.predictions.flatten() , p.label_ids.flatten()
    mask = labels!=-100
    logits , labels = logits[mask] , labels[mask]
    print(classification_report(labels, logits, digits=9))
    report_output = classification_report(labels, logits, digits=9, output_dict=True)
    metrics_dic = {
      f'accuracy' : accuracy_score(labels, logits),
      f'f1_score_avg' : report_output['accuracy'],
      f'f1_score_macro' : report_output['macro avg']['f1-score'],
      f'f1_score_weighted' : report_output['weighted avg']['f1-score'],
    }
    return metrics_dic

class NerLevelTrainer(Trainer):

    loss_fct = torch.nn.CrossEntropyLoss()

    def compute_loss(self,model,inputs,return_outputs=False):
        logits = model(**inputs).logits
        labels = inputs['labels']
        # Only keep active parts of the loss
        active_loss = inputs['attention_mask'].view(-1) == 1
        active_logits = logits.view(-1, model.config.num_labels)
        active_labels = torch.where(active_loss, labels.view(-1), torch.tensor(self.loss_fct.ignore_index).type_as(labels))
        non_padded_mask = active_labels!=-100
        loss = self.loss_fct(active_logits[non_padded_mask], active_labels[non_padded_mask])
        active_preds_mask = torch.logical_and(active_loss,non_padded_mask)
        return (loss,(loss,torch.argmax(logits,2))) if return_outputs else loss

label_names = ['labels']
args = TrainingArguments(
        output_dir = '/content',
        save_total_limit = 1,
        evaluation_strategy = "epoch",
        learning_rate = lr,
        per_device_train_batch_size=bs,
        per_device_eval_batch_size=bs,
        remove_unused_columns=False,
        num_train_epochs = 15,
        weight_decay = 0.01,
        save_strategy = 'epoch',
        load_best_model_at_end=True,
        metric_for_best_model='f1_score_avg',
        label_names=label_names
    )

trainer = NerLevelTrainer(
    model=model_,
    args=args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['validation'],
    compute_metrics=compute_metrics
    )

early_stopping = EarlyStoppingCallback(early_stopping_patience = 3)
trainer.add_callback(early_stopping)
trainer.train()
trainer.evaluate(dataset_dict['test'])
pd.DataFrame(trainer.state.log_history).to_csv(f'results/nergedfuse_adapters_distilroberta_lr{lr}_bs{bs}_norm.csv')

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for prediction

Epoch,Training Loss,Validation Loss,Accuracy,F1 Score Avg,F1 Score Macro,F1 Score Weighted
1,No log,0.207836,0.929169,0.929169,0.569167,0.914547
2,No log,0.149375,0.955999,0.955999,0.757420,0.955453
3,No log,0.148396,0.950450,0.950450,0.734574,0.945606
4,0.229600,0.140461,0.954052,0.954052,0.756481,0.950403
5,0.229600,0.115663,0.964507,0.964507,0.802043,0.963072
6,0.229600,0.125410,0.963241,0.963241,0.805836,0.963185
7,0.229600,0.114735,0.963319,0.963319,0.806049,0.962054
8,0.135300,0.115866,0.962988,0.962988,0.796069,0.959999


***** Running Evaluation *****
  Num examples = 266
  Batch size = 8


              precision    recall  f1-score   support

           0  0.953071176 0.996468580 0.974286858     42759
           1  0.797758533 0.850162866 0.823127464      1842
           2  0.927835052 0.895179801 0.911214953      1307
           3  0.715160796 0.348247576 0.468405216      1341
           4  0.619651347 0.520639148 0.565846599       751
           5  0.706488157 0.746869897 0.726118021      1837
           6  0.709459459 0.408560311 0.518518519       257
           7  0.750000000 0.035791757 0.068322981       922
           8  0.857142857 0.034682081 0.066666667       346

    accuracy                      0.929169425     51362
   macro avg  0.781840820 0.537400224 0.569167475     51362
weighted avg  0.921442547 0.929169425 0.914547159     51362



Saving model checkpoint to /content/checkpoint-133
Configuration saved in /content/checkpoint-133/ged/adapter_config.json
Module weights saved in /content/checkpoint-133/ged/pytorch_adapter.bin
Configuration saved in /content/checkpoint-133/ged/adapter_fusion_config.json
Module weights saved in /content/checkpoint-133/ged/pytorch_model_adapter_fusion.bin
***** Running Evaluation *****
  Num examples = 266
  Batch size = 8


...

              precision    recall  f1-score   support

           0  0.983566070 0.996585514 0.990032991     42759
           1  0.979226774 0.921281216 0.949370629      1842
           2  0.983695652 0.969395562 0.976493256      1307
           3  0.667040359 0.887397465 0.761600000      1341
           4  0.798657718 0.792276964 0.795454545       751
           5  0.952610442 0.645617855 0.769630110      1837
           6  0.855614973 0.622568093 0.720720721       257
           7  0.704439252 0.654013015 0.678290214       922
           8  0.839195980 0.482658960 0.612844037       346

    accuracy                      0.963319185     51362
   macro avg  0.862671913 0.774643849 0.806048500     51362
weighted avg  0.964715428 0.963319185 0.962053529     51362



Saving model checkpoint to /content/checkpoint-931
Configuration saved in /content/checkpoint-931/ged/adapter_config.json
Module weights saved in /content/checkpoint-931/ged/pytorch_adapter.bin
Configuration saved in /content/checkpoint-931/ged/adapter_fusion_config.json
Module weights saved in /content/checkpoint-931/ged/pytorch_model_adapter_fusion.bin
Deleting older checkpoint [/content/checkpoint-798] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 266
  Batch size = 8


              precision    recall  f1-score   support

           0  0.975407401 0.998082275 0.986614574     42759
           1  0.966648138 0.944082519 0.955232079      1842
           2  0.984459984 0.969395562 0.976869699      1307
           3  0.783333333 0.806114840 0.794560823      1341
           4  0.818051576 0.760319574 0.788129745       751
           5  0.904607398 0.758845944 0.825340438      1837
           6  0.840796020 0.657587549 0.737991266       257
           7  0.775043937 0.478308026 0.591549296       922
           8  0.910447761 0.352601156 0.508333333       346

    accuracy                      0.962988201     51362
   macro avg  0.884310616 0.747259716 0.796069028     51362
weighted avg  0.960767896 0.962988201 0.959998757     51362



Saving model checkpoint to /content/checkpoint-1064
Configuration saved in /content/checkpoint-1064/ged/adapter_config.json
Module weights saved in /content/checkpoint-1064/ged/pytorch_adapter.bin
Configuration saved in /content/checkpoint-1064/ged/adapter_fusion_config.json
Module weights saved in /content/checkpoint-1064/ged/pytorch_model_adapter_fusion.bin
Deleting older checkpoint [/content/checkpoint-931] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best adapter(s) from /content/checkpoint-665 (score: 0.9645068338460341).
Loading module configuration from /content/checkpoint-665/ged/adapter_config.json
Overwriting existing adapter 'ged'.
Loading module weights from /content/checkpoint-665/ged/pytorch_adapter.bin
No matching prediction head found in '/content/checkpoint-665/ged'
Loading best adapter fusion(s) from /content/checkpoint-665 (score: 0.9645068338460341).
Loading module configuration from /cont

              precision    recall  f1-score   support

           0  0.988041076 0.991702111 0.989868209     38323
           1  0.977631579 0.918985776 0.947401976      1617
           2  0.980530973 0.958477509 0.969378828      1156
           3  0.785090274 0.811559302 0.798105388      1661
           4  0.766304348 0.844311377 0.803418803       835
           5  0.828062105 0.863309353 0.845318462      1668
           6  0.709433962 0.731517510 0.720306513       257
           7  0.743697479 0.504273504 0.601018676       702
           8  0.620689655 0.583333333 0.601431981       216

    accuracy                      0.963928071     46435
   macro avg  0.822164606 0.800829975 0.808472093     46435
weighted avg  0.963553312 0.963928071 0.963297349     46435

